<a href="https://colab.research.google.com/github/Abeszz/SC4002-NLP-Assignment/blob/main/SC4002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Requirements :

In [1]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00


In [2]:
pip install nltk

In [4]:
pip install numpy

In [9]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-10-14 17:24:42--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-14 17:24:43--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2024-10-14 17:27:23 (5.16 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [23]:
import os
import nltk
import sys

env_base_path = sys.prefix
nltk_path = os.path.join(env_base_path, 'nltk_data')
nltk.download('punkt', nltk_path)
nltk.download('punkt_tab', nltk_path)

[nltk_data] Downloading package punkt to /usr/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /usr/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# **PART 0. Dataset Preparation**

In [6]:
from datasets import load_dataset
dataset = load_dataset('rotten_tomatoes')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [7]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

# **Part 1. Preparing Word Embedding**

In [37]:
vocabulary = None
vocab_size = None
embedding_matrix = None
oov_words = [] # Array to track OOV words
UNKNOWN_TOKEN = '<UNKNOWN>'  # Special token for OOV words

glove_model_file = 'glove.6B.300d.txt'
glove_model_dimension = 300

In [38]:
from nltk.tokenize import word_tokenize
from collections import Counter

def tokenize_sentence(sentence):
    # Tokenize each sentence using NLTK's word_tokenize
    return word_tokenize(sentence.lower()) # Convert to lowercase for consistency

def tokenize_dataset(dataset):
    global vocabulary, vocab_size
    # Build a vocabulary from the dataset
    vocab_counter = Counter()

    for word in dataset:
        tokens = tokenize_sentence(word['text'])
        vocab_counter.update(tokens)

    # Update the vocabulary list
    vocabulary = list(vocab_counter.keys())
    vocab_size = len(vocabulary)

In [39]:
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

In [40]:
import numpy as np

def create_embedding_matrix(glove_file_path, glove_model_dimension):
    global vocabulary, embedding_matrix, oov_words
    # Load the GloVe embeddings into a dictionary
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create the embedding matrix
    embedding_matrix = np.zeros((vocab_size, glove_model_dimension))  # Initialize the matrix with zeros

    # Initialize the UNKNOWN token with a random vector
    unknown_vector = np.random.normal(scale=0.6, size=(glove_model_dimension,))

    # Fill the embedding matrix
    for idx, word in enumerate(vocabulary):
        if word in glove_embeddings:
            embedding_matrix[idx] = glove_embeddings[word]  # Use the pretrained GloVe vector
        elif word == UNKNOWN_TOKEN:
            embedding_matrix[idx] = unknown_vector  # Do not add UNKNOWN token to oov
        else:
            oov_words.append(word)  # Track OOV words
            embedding_matrix[idx] = unknown_vector

In [41]:
def add_unkown_token():
    global vocabulary, vocab_size, UNKNOWN_TOKEN

    # Check if 'UNK' is already in the vocabulary, if not, add it
    if UNKNOWN_TOKEN not in vocabulary:
        vocabulary.append(UNKNOWN_TOKEN)
        vocab_size += 1


In [42]:
# Tokenize the dataset
tokenize_dataset(train_dataset)

In [43]:
# Print the size of Vocabulary
print(f"1(a) Number of Vocabulary words in training dataset = {vocab_size}")

1(a) Number of Vocabulary words in training dataset = 18029


In [44]:
# 1(c) : Add <UNKNOWN> token and replace all OOV words with it
add_unkown_token()

In [45]:
# Initialize the glove model with our configuration
create_embedding_matrix(glove_model_file, glove_model_dimension)

In [46]:
# Print the number of OOV words
print(f"1(b) Number of Out Of Vocabulary words: {len(oov_words)}")

1(b) Number of Out Of Vocabulary words: 1865
